# Sentence Emotion Detection Model

Code used to make sentiment analysis model

In [ ]:
!pip install --upgrade torch==1.7.1 torchtext==0.8.1 torchvision==0.8.2

     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 7.0MB 21.2MB/s 
     |████████████████████████████████| 12.8MB 245kB/s 
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101


In [ ]:
import torch, torchtext
from torch import nn, optim, functional as F
import pandas as pd, csv
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pdb
import random

In [ ]:
!wget -O text.csv https://www.dropbox.com/s/iulhdbo1yc8farq/Emotion_final.csv?dl=0

--2021-04-12 04:45:40--  https://www.dropbox.com/s/iulhdbo1yc8farq/Emotion_final.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/iulhdbo1yc8farq/Emotion_final.csv [following]
--2021-04-12 04:45:40--  https://www.dropbox.com/s/raw/iulhdbo1yc8farq/Emotion_final.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce4852207e46ddab5c245a258cf.dl.dropboxusercontent.com/cd/0/inline/BMfDqv21X_gsA8kE7TAG77WqffYflzYy4hhiwxGIN60ZpzfxCqXxExbvEV_0BPD648NVaTBjsxULo7CgBR3YiW7t1g5s7savpJnD6hHFTjbsTQRfYgj-bAqyxDon-rJElHz2T-Te007SojmTAfjYuJhD/file# [following]
--2021-04-12 04:45:40--  https://uce4852207e46ddab5c245a258cf.dl.dropboxusercontent.com/cd/0/inline/BMfDqv21X_gsA8kE7TAG77WqffYflzYy4hhiwxGIN60ZpzfxCqXxExbvEV_0BPD648NVaT

In [ ]:
text = pd.read_csv('/content/text.csv')

In [ ]:
sentiment = ['sadness', 'anger', 'love', 'surprise', 'fear', 'happy']

In [ ]:
class Sentences(torch.utils.data.Dataset):
    def __init__(self, fn):
        lengths = []
        convert = { u: n for n, u in enumerate(fn['Emotion'].unique()) }
        fn['Emotion'] = fn['Emotion'].apply(lambda u: convert[u])               # 12 unique words should be assigned integers starting from 0
        tokenizer = torchtext.data.utils.get_tokenizer('spacy', 'en_core_web_sm')# tokenizer using spaCy
        for i in range(len(fn['Text'])):
          lengths.append(len(tokenizer(fn['Text'].iat[i].strip())))                   # store the number of tokens in each sentence to beused in get item
        string = ' '.join([fn['Text'].iat[i].strip() 
                           for i in range(len(fn['Text']))])                  # combine everything into one single string
        toks = tokenizer(string)                                                # tokenize the single string

        self.vocab = torchtext.vocab.build_vocab_from_iterator([toks])
        self.sentiment = fn['Emotion'].values
        self.text = fn['Text'].values
        self.length = lengths
        self.toks = torch.LongTensor([self.vocab[tok] for tok in toks])

    def __len__(self):
        return len(self.length)

    def __getitem__(self, i):
        sum = 0
        for x in range(i):
          sum += self.length[x]
        return (self.sentiment[i], self.toks[sum: sum + self.length[i]])          # return the sentiment and related tokns for a specific tweet

In [ ]:
ds_full = Sentences(text)
n_train = int(0.8 * len(ds_full))
n_test = len(ds_full) - n_train
rng = torch.Generator().manual_seed(291)
ds_train, ds_test = torch.utils.data.random_split(ds_full, [n_train, n_test], rng)

1lines [00:00, 20.43lines/s]


In [ ]:
class SentenceModel(nn.Module):                                                 # takes in a sentence, and outputs predicted sentiment
      def __init__(self, vocab_size, embedding_dim, lstm_dim, 
                   n_cats, n_layers = 2, drop_prob = 0.5):
        super().__init__()                                                      #constructor for parent class
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)          #use word embeddings 
        self.lstm = torch.nn.LSTM(embedding_dim, lstm_dim, n_layers,
                                  dropout=drop_prob, batch_first=True)          #LSTM layer
        self.linear = nn.Linear(lstm_dim, n_cats)
        nn.init.xavier_uniform_(self.embedding.weight.data)
        nn.init.xavier_uniform_(self.linear.weight.data)
        
      def forward(self, text):
        emb = self.embedding(text)
        lstm_out, _ = self.lstm(emb)
        out = self.linear(lstm_out)
        return torch.mean(out, dim=1)                                           # certain dimensions required so take mean to reduce them down

In [ ]:
device = torch.device('cpu')

def run_test(model, ds, crit):
    preds = []                                                                  # array to store predictions
    batch_size = 1                                                              # change batch size here
    model.eval()
    total_loss, total_acc = 0, 0
    ldr = torch.utils.data.DataLoader(ds)
    for labs, txts in ldr:                                                
        labs, txts = labs.to(device), txts.to(device)
        with torch.no_grad():
            outs = model(txts)
            loss = crit(outs, labs)
            total_loss += loss.item()
            total_acc += (outs.argmax(1) == labs).sum().item()
            preds.append(outs.argmax(1))                                        # append all the predictions to an array
    return total_loss / len(ds), total_acc / len(ds), preds, batch_size         # added array return value 'preds' and batchsize

def run_train(model, ds, crit, opt, sched):
    model.train()
    total_loss, total_acc = 0, 0
    ldr = torch.utils.data.DataLoader(ds)
    for labs, txts in ldr:          
        opt.zero_grad()
        labs, txts = labs.to(device), txts.to(device)
        outs = model(txts)                                                      
        loss = crit(outs, labs)
        loss.backward()
        opt.step()
        total_loss += loss.item()
        total_acc += (outs.argmax(1) == labs).sum().item()
    sched.step()
    return total_loss / len(ds), total_acc / len(ds)

def run_all(model, test_ds, train_ds, crit, opt, sched, n_epochs=10):
    for epoch in tqdm(range(n_epochs), desc='epochs'):
        train_loss, train_acc = run_train(model, train_ds, crit, opt, sched)
        test_loss, test_acc, _, _ = run_test(model, test_ds, crit)
        tqdm.write(f'epoch {epoch}   train loss {train_loss:.6f} acc {train_acc:.4f}   test loss {test_loss:.6f} acc {test_acc:.4f}')   

In [ ]:
model = SentenceModel(len(ds_full.vocab), 64, 64, len(text.Emotion.unique()))
device = torch.device('cuda:0')
model.to(device);
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=0.1) #lr: 1 -> 0.1 (counteract the loss increase over time)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=1)

run_all(model, ds_test, ds_train, crit, opt, sched, 20)

epoch 0   train loss 1.608359 acc 0.3168   test loss 1.583686 acc 0.3269
epoch 1   train loss 1.606533 acc 0.3162   test loss 1.583427 acc 0.3269
epoch 2   train loss 1.606486 acc 0.3162   test loss 1.583368 acc 0.3269
epoch 3   train loss 1.606475 acc 0.3161   test loss 1.583344 acc 0.3269
epoch 4   train loss 1.606457 acc 0.3162   test loss 1.583283 acc 0.3269
epoch 5   train loss 1.606383 acc 0.3161   test loss 1.583106 acc 0.3269
epoch 6   train loss 1.605800 acc 0.3163   test loss 1.582347 acc 0.3269
epoch 7   train loss 1.580952 acc 0.3430   test loss 1.643022 acc 0.4007
epoch 8   train loss 1.259339 acc 0.5179   test loss 1.073486 acc 0.5874
epoch 9   train loss 0.934875 acc 0.6407   test loss 0.759820 acc 0.7216
epoch 10   train loss 0.656265 acc 0.7604   test loss 0.521467 acc 0.8197
epoch 11   train loss 0.425697 acc 0.8469   test loss 0.382146 acc 0.8681
epoch 12   train loss 0.303684 acc 0.8896   test loss 0.297928 acc 0.8947
epoch 13   train loss 0.227618 acc 0.9138   test